# Introduction

<p>In a United Parcel Service (UPS) package sorting facility, also known as a hub, packages that are not large enough to be sorted individually on the main conveyor belts are automatically sent to an area of the hub known as “small sort” to become “containerized.” This is the process of combining packages that have similar service levels and similar final delivery destinations into bags so that the bag can be routed to its delivery vehicle. The list of packages destined for the same bag will be referred to as a “load.”</p>
&nbsp;&nbsp;
<p>In most UPS hubs, a tilt-tray conveyor system is used to drop packages into the correct bags. When a bag becomes full, an employee needs to promptly zip up the bag, print out a label, place the label on the bag, place the bag onto a conveyor belt where it will be routed to the appropriate delivery vehicle, and place an empty bag in its place so that more packages can be dropped in that bin position. A single position alongside the tilt tray where packages are dropped, and bags are placed to catch them will be referred to as a “bin position.” The goal of this project is to present a mathematical model known as an assignment linear program for assigning loads to bin position in a manner that maximizes the productivity of the employees processing the volume.</p>


# Assignment Linear Program Formulation

<p>An assignment linear programming (LP) problem is a type of optimization problem where the goal is to assign a set of tasks to a set of agents to minimize or maximize a linear objective function that is subject to linear constraints. In the load-to-bin assignment LP problem, the objective function is the sum of the cost for every assignment made. Hence, the goal is to minimize this function while satisfying the constraints that every load be assigned to one bin position and every bin position be assigned one load.</p>
&nbsp;&nbsp;


### Assumptions and Initial Conditions

1. The information needed to model the problem is a list of loads that are awaiting a bin assignment and the corresponding average daily package volume for each of these loads. A list of the available bins is also needed for the purpose of knowing how many bins are available.
&nbsp;&nbsp;
2. The model assumes that the number of loads that need to be assigned is equal to the number of bins available. In the case where the number of loads awaiting assignment is less than the number of bins available, “dummy” loads are created that receive zero package volume to make the number of loads equal the number of bins. In the case where the number of loads awaiting assignment is greater than the number of bins available, the problem is declared to not be feasible, and the user will need to arbitrarily combine loads together to reduce the number of loads.
&nbsp;&nbsp;
3. This formulation assumes that the number of employees working in the tilt-tray operations is known prior to solving the problem. Employees are assumed to be spread out along the tilt tray with equal distance from each other.


#Step 1: Generate a Dataframe for Loads

### Columns:

1. Load Name

  * Contains a list of load names.

2. AVG Daily Volume

  * Contains the average daily package volume for each load.
  * In practice, this data can be obtained from a UPS corporate database by a UPS industrial engineer but is generated randomly from a uniform distribution in this project for the purpose of data privacy.

In [7]:
import numpy as np
import pandas as pd
import random

# Parameters
num_loads = 100       # Number of loads needing bin assignment
low = 0               # Lowest Possible AVG Daily Volume in uniform distribution
high = 200            # Highest Possible AVG Daily Volume in uniform distribution

# Generate Column 1: Load Names
load_names = [f"Load {i+1}" for i in range(num_loads)]

# Generate Column 2: AVG Daily Volume
avg_daily_volume = [round(random.uniform(low, high),2) for _ in range(num_loads)]

# Create the DataFrame
loads_df = pd.DataFrame({
    "Load Name": load_names,
    "AVG Daily Volume": avg_daily_volume,
})

loads_df

,Load Name,AVG Daily Volume
0,Load 1,56.23
1,Load 2,121.33
2,Load 3,161.63
3,Load 4,110.03
4,Load 5,117.88
...,...,...
95,Load 96,138.77
96,Load 97,67.78
97,Load 98,51.40
98,Load 99,39.33


# Step 2: Generate a Dataframe for Bin Positions

### Columns:

1. Bin Position Name

  * This column contains a list of bin position names.

2. Home Position (Y/N)

  * A "$1$" in this column indicates that the corresponding bin positon (in the same row) is a "home position" for a UPS small sort employee.
  * A "$0$" indicates that the corresponding bin position is not a home position.
  * Home positions are spaced out evenly across the space of bin positions.
  * The number of home positions is equal to the number of small sort employees.

3. Cost Multiplier

  * The value in this column represents the distance (in terms of number of bin positions) that the corresponding bin position is from a home position
  * These values are important in the next step of model, creating a cost coefficeint matrix


In [10]:
import numpy as np
import pandas as pd

# Parameters
num_bins = num_loads  # Number of bins is assumed to be equal to the number of loads needing bin assignment
num_employees = 5     # Number of employees to be spaced evenly along tilt-tray

# Generate Column 1: Bin Position Names
binposition_names = [f"Bin Position {i+1}" for i in range(num_bins)]

# Generate Column 2: Home Position (Y/N)
indicator = np.zeros(num_bins, dtype=int)
interval = num_bins // num_employees
start_point = (num_bins - (interval * num_employees)) // 2

for i in range(num_employees):
    position = start_point + i * interval + interval // 2
    indicator[position] = 1

# Generate Column 3: Cost Multiplier
multiplier = np.zeros(num_bins, dtype=int)
for i in range(num_bins):
    if indicator[i] == 1:
        multiplier[i] = 0
    else:
        distances = []
        for j in range(num_bins):
            if indicator[j] == 1:
                distances.append(abs(j - i))
        multiplier[i] = min(distances)

# Create the DataFrame
bins_df = pd.DataFrame({
    "Bin Position Name": binposition_names,
    "Home Position (Y/N)": indicator,
    "Cost Multiplier": multiplier
})

bins_df

,Bin Position Name,Home Position (Y/N),Cost Multiplier
0,Bin Position 1,0,10
1,Bin Position 2,0,9
2,Bin Position 3,0,8
3,Bin Position 4,0,7
4,Bin Position 5,0,6
...,...,...,...
95,Bin Position 96,0,5
96,Bin Position 97,0,6
97,Bin Position 98,0,7
98,Bin Position 99,0,8


# Creating a Cost Cost Coefficent Matrix


In [25]:
# Create Cost Coefficent Matrix

load_vol = loads_df['AVG Daily Volume']
multipliers = bins_df['Cost Multiplier']

matrix = np.zeros((num_loads, num_bins))

for i in range(num_loads):
    for j in range(num_bins):
      matrix[i, j] = multiplier[i]*load_vol[j]

cost_coefficients_df = pd.DataFrame(matrix, columns=[f'Load {i+1}' for i in range(num_loads)], index=[f'Bin {i+1}' for i in range(num_bins)])

cost_coefficients_df.head(10)






,Load 1,Load 2,Load 3,Load 4,Load 5,Load 6,Load 7,Load 8,Load 9,Load 10,...,Load 91,Load 92,Load 93,Load 94,Load 95,Load 96,Load 97,Load 98,Load 99,Load 100
Bin 1,562.30,1213.30,1616.30,1100.30,1178.80,947.0,1226.20,125.40,343.30,1987.10,...,1946.40,317.80,942.0,1783.90,1585.80,1387.70,677.80,514.0,393.30,1480.90
Bin 2,506.07,1091.97,1454.67,990.27,1060.92,852.3,1103.58,112.86,308.97,1788.39,...,1751.76,286.02,847.8,1605.51,1427.22,1248.93,610.02,462.6,353.97,1332.81
Bin 3,449.84,970.64,1293.04,880.24,943.04,757.6,980.96,100.32,274.64,1589.68,...,1557.12,254.24,753.6,1427.12,1268.64,1110.16,542.24,411.2,314.64,1184.72
Bin 4,393.61,849.31,1131.41,770.21,825.16,662.9,858.34,87.78,240.31,1390.97,...,1362.48,222.46,659.4,1248.73,1110.06,971.39,474.46,359.8,275.31,1036.63
Bin 5,337.38,727.98,969.78,660.18,707.28,568.2,735.72,75.24,205.98,1192.26,...,1167.84,190.68,565.2,1070.34,951.48,832.62,406.68,308.4,235.98,888.54
Bin 6,281.15,606.65,808.15,550.15,589.40,473.5,613.10,62.70,171.65,993.55,...,973.20,158.90,471.0,891.95,792.90,693.85,338.90,257.0,196.65,740.45
Bin 7,224.92,485.32,646.52,440.12,471.52,378.8,490.48,50.16,137.32,794.84,...,778.56,127.12,376.8,713.56,634.32,555.08,271.12,205.6,157.32,592.36
Bin 8,168.69,363.99,484.89,330.09,353.64,284.1,367.86,37.62,102.99,596.13,...,583.92,95.34,282.6,535.17,475.74,416.31,203.34,154.2,117.99,444.27
Bin 9,112.46,242.66,323.26,220.06,235.76,189.4,245.24,25.08,68.66,397.42,...,389.28,63.56,188.4,356.78,317.16,277.54,135.56,102.8,78.66,296.18
Bin 10,56.23,121.33,161.63,110.03,117.88,94.7,122.62,12.54,34.33,198.71,...,194.64,31.78,94.2,178.39,158.58,138.77,67.78,51.4,39.33,148.09


In [31]:
!pip install pulp
from pulp import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 45.1 MB/s eta 0:00:00


In [42]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Assume cost_coefficients_df has been defined and has the appropriate costs.

# Define the problem
prob = LpProblem("AssignmentProblem", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("x", [(i, j) for i in range(num_bins) for j in range(num_loads)], cat='Binary')

# Objective function: Minimize the total cost
prob += lpSum([cost_coefficients_df.iloc[i, j] * x[(i, j)] for i in range(num_bins) for j in range(num_loads)])

# Constraints: Each bin is assigned to exactly one load
for i in range(num_bins):
    prob += lpSum([x[(i, j)] for j in range(num_loads)]) == 1

# Constraints: Each load is assigned to exactly one bin
for j in range(num_loads):
    prob += lpSum([x[(i, j)] for i in range(num_bins)]) == 1

# Solve the problem
prob.solve()

# Extract the assignments
assignments = [(i, j) for i in range(num_bins) for j in range(num_loads) if x[(i, j)].varValue == 1]

# Create a DataFrame to display the assignments
result_df = pd.DataFrame(assignments, columns=['Bin', 'Load'])

# Add Bins column with numbers counting from 1 to num_bins
result_df['Bin Position'] = result_df['Bin'] + 1

# Add Load names and Avg Daily Volume from the loads_df
result_df['Load Name'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'Load Name'])
result_df['AVG Daily Volume'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'AVG Daily Volume'])

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'AVG Daily Volume']]

# Create horizontal bars starting from the left
max_length = 20  # Maximum length of the bar
max_value = result_df['AVG Daily Volume'].max()
result_df['Bar'] = result_df['AVG Daily Volume'].apply(lambda x: '█' * int(max_length * (x / max_value)))

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'AVG Daily Volume', 'Bar']]

result_df


,Bin Position,Load Name,AVG Daily Volume,Bar
0,1,Load 73,11.03,█
1,2,Load 8,12.54,█
2,3,Load 12,51.63,█████
3,4,Load 37,65.58,██████
4,5,Load 61,82.13,████████
...,...,...,...,...
95,96,Load 11,111.58,███████████
96,97,Load 6,94.70,█████████
97,98,Load 55,74.69,███████
98,99,Load 53,44.76,████
